<a href="https://colab.research.google.com/github/spatialthoughts/projects/blob/master/ee-python/dynamic_visualization_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computing Dynamic Visualization Parameters 

Notebook  showing how to dynamically compute min/max values from an image and use it for visualization.

In [1]:
import ee

In [3]:
# Replace the cloud_project with your own project
cloud_project = 'spatialthoughts'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

In [7]:
try:
    import geemap
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print('geemap not found, installing via pip in Google Colab...')
        !pip install --quiet geemap
        import geemap
    else:
        print('geemap not found, please install via conda in your environment')

Load Terra Land Surface Temperature and Emissivity 8-Day Global 1km collection and filter to April 2022 when Indian sub-continent was experiencing unprecedented heat-wave.

In [8]:
dataset = ee.ImageCollection('MODIS/061/MOD11A2') \
  .filter(ee.Filter.date('2022-04-20', '2022-04-30'))
# Select LST_Day_1km band
landSurfaceTemperature = dataset.select('LST_Day_1km')

map() a function to apply scaling and conversion from Kelvin to Celsius.

In [11]:
def scaleAndConvert(image):
  return image \
    .multiply(0.02) \
    .subtract(273.15) \
    .copyProperties(image, ['system:time_start'])

modis_LST = landSurfaceTemperature.map(scaleAndConvert)


Extract an image.

In [14]:
LST_Mosaic = ee.Image(modis_LST.first())

Define a color palette and bounding box.

In [15]:
# Define a color palette
palette = [
  '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
  '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
  '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
  'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
  'ff0000', 'de0101', 'c21301', 'a71001', '911003'
]

# Construct a global polygon that we can use to compute stats
geometry = ee.Algorithms.GeometryConstructors.BBox(-180, -90, 180, 90)

We now compute the image statistics. We can use a lower scale to compute approximate stats. Since we want to apply the visualization parameters dynamically speed is more important than accuracy.

In [17]:
stats = LST_Mosaic.reduceRegion(**{
  'reducer': ee.Reducer.minMax(),
  'geometry': geometry,
  'scale': 10000,
  'maxPixels': 1e10
})

min = stats.get('LST_Day_1km_min').getInfo()
max = stats.get('LST_Day_1km_max').getInfo()
print('Min', min)
print('Max', max)

Min -80.88999999999999
Max 60.47000000000003


Now that we have dynamically computed the minimum and maximum pixel value from the image, we can use it to visualize the image and load it on the map.

In [19]:
Map = geemap.Map()

landSurfaceTemperatureVis = {
    'min': min,
    'max': max,
    'palette': palette
  }

Map.setCenter(75.74, 24.89, 4)
Map.addLayer(LST_Mosaic, landSurfaceTemperatureVis, 'Land Surface Temperature')
Map

Map(center=[24.89, 75.74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…